# Image Processing Pipeline for Cell Segmentation Training

This notebook is the **first step** of a two-part protocol for automated cell counting in fluorescence microscopy images.

## Overview
This code processes .czi microscopy files by:
1. **Splitting multi-channel images**: Extracts individual fluorescence channels (C0, C1, C2, C3) and Z-planes from .czi files
2. **Converting to OME-TIFF format**: Saves each channel/Z-plane combination as individual .tif files for easier processing
3. **Automated cell segmentation**: Uses Cellpose (cpsam model) to automatically detect cell boundaries and create segmentation masks

## Workflow
- **Input**: .czi files from confocal microscopy
- **Output**: Individual channel images + segmentation masks in a "train" folder
- **Next step**: Run `Contar_Rita2.ipynb` to perform cell counting and co-localization analysis

## Files Generated
- `filename_C0_Z0.ome.tif`: Channel 0, Z-plane 0 image
- `filename_C0_Z0.ome_mask.tiff`: Corresponding segmentation mask
- `filename_C0_Z0.zip`: ROI coordinates for detected cells

In [12]:
# Import required libraries for image processing
from aicspylibczi import CziFile  # For reading .czi microscopy files
from tifffile import imwrite      # For writing TIFF images
from pathlib import Path          # For file path operations
import numpy as np                # For numerical operations

def export_channels_z(czi_file_list):
    """
    Step 1: Extract Individual Channels and Z-planes from CZI Files
    
    This function takes .czi microscopy files and splits them into individual
    channel/Z-plane combinations, saving each as a separate OME-TIFF file.
    
    Parameters:
    - czi_file_list: List of paths to .czi files to process
    
    Output:
    - Creates a 'train' folder next to each original file
    - Saves individual channel/Z-plane images as OME-TIFF files
    - Naming convention: filename_C{channel}_Z{z-plane}.ome.tif
    """
    # Ensure input is always a list for consistent processing
    if isinstance(czi_file_list, (str, Path)):
        czi_file_list = [czi_file_list]

    # Process each .czi file in the list
    for file in czi_file_list:
        file_path = Path(file)
        
        # Check if file exists before processing
        if not file_path.exists():
            print(f"[ERROR] File not found: {file_path}")
            continue

        try:
            print(f"[INFO] Processing: {file_path.name}")
            
            # Open the .czi file using aicspylibczi
            czi = CziFile(file_path)

            # Create output directory for processed images
            output_folder = file_path.parent / "train"
            output_folder.mkdir(parents=True, exist_ok=True)

            # Extract base filename without extension
            base_name = file_path.stem

            # Get image dimensions to determine number of channels and Z-planes
            dims = czi.get_dims_shape()[0]
            num_channels = dims['C'][1]  # Number of fluorescence channels
            num_z = dims['Z'][1]         # Number of Z-planes (depth)

            print(f"[INFO] Found {num_channels} channels and {num_z} Z-planes")

            # Extract each channel/Z-plane combination
            for c in range(num_channels):
                for z in range(num_z):
                    # Read the specific channel and Z-plane
                    # Convert to 16-bit unsigned integer for compatibility
                    img = czi.read_mosaic(C=c, Z=z).squeeze().astype(np.uint16)
                    
                    # Generate filename following naming convention
                    tif_name = f"{base_name}_C{c}_Z{z}.ome.tif"
                    tif_path = output_folder / tif_name

                    # Save as OME-TIFF with proper metadata
                    imwrite(
                        tif_path,
                        img,
                        photometric="minisblack",  # Grayscale format
                        metadata={"axes": "YX"}    # 2D image format
                    )

            print(f"[OK] Export complete: {file_path.name}")

        except Exception as e:
            print(f"[ERROR] Failed to process {file_path.name}: {e}")

# Define files to process
# Add your .czi file paths here
files = [
    "../slice_8/20240913_rc_brain_2_3_fos_neun_gfp-01_AcquisitionBlock8_pt8-Stitching-08.czi",
    # Add more files as needed:
    # "/path/to/another/file.czi"
]

# Execute the channel extraction
export_channels_z(files)


[INFO] Processing: 20240913_rc_brain_2_3_fos_neun_gfp-01_AcquisitionBlock8_pt8-Stitching-08.czi
[OK] Export complete: 20240913_rc_brain_2_3_fos_neun_gfp-01_AcquisitionBlock8_pt8-Stitching-08.czi


In [8]:
# Step 2: Automated Cell Segmentation using Cellpose
# Import required libraries for cell segmentation
from cellpose import models, io    # Cellpose for automated cell segmentation
import matplotlib.pyplot as plt   # For plotting (if needed)
import skimage.io as skio          # For image input/output
from pathlib import Path           # For file path operations
import time                        # For timing the segmentation process
import os                          # For file system operations
import numpy as np                 # For numerical operations

# Define the folder containing the extracted channel images
# This should point to the 'train' folder created in Step 1
output_folder = Path("../slice_8/train")  # Update this path as needed

# Find all .tif images (channel images) and existing mask files
images = sorted(output_folder.glob('*.tif'))
existing_masks = sorted(output_folder.glob('*_masks.tif'))

print(f"[INFO] Found {len(images)} images to process")
print(f"[INFO] Found {len(existing_masks)} existing masks (will skip these)")

# Initialize Cellpose model
# 'cpsam' is a specialized model for cell segmentation
# gpu=True enables GPU acceleration (set to False if no GPU available)
model = models.CellposeModel(pretrained_model='cpsam', gpu=True)
print("[INFO] Cellpose model loaded successfully")

# Process each image for cell segmentation
for img_path in images:
    # Check if mask already exists to avoid reprocessing
    mask_name = img_path.stem + '_masks.tif'
    if not any(mask_name in str(f) for f in existing_masks):
        print(f"[INFO] Processing: {img_path.name}")
        
        # Start timing
        t0 = time.time()
        
        # Load the image
        image_data = skio.imread(img_path)
        
        # Run Cellpose segmentation
        masks, flows, _ = model.eval(
            image_data,
            diameter=None,           # Auto-adjust for cell size
            flow_threshold=0.4,      # Threshold for flow quality (0-1)
            cellprob_threshold=0.0,  # Threshold for cell probability (0-1)
            do_3D=False              # Set to True for 3D images
        )
        
        # Save segmentation mask as TIFF
        output_path = output_folder / f'{img_path.stem}_mask.tiff'
        skio.imsave(output_path, masks.astype(np.uint16))
        
        # Save ROI coordinates as ZIP file for ImageJ compatibility
        roi_path = output_folder / f"{img_path.stem}.zip"
        io.save_rois(masks, str(roi_path))
        
        # Print processing time
        elapsed_time = time.time() - t0
        print(f"[OK] Completed in {elapsed_time:.2f}s: {img_path.stem}")
        
    else:
        print(f"[SKIP] Mask already exists for: {img_path.name}")


/home/bioimaging/miniconda3/envs/cellpose/lib/python3.10/site-packages/skimage/_shared/utils.py:328: UserWarning: /media/bioimaging/SWAP/Rita_Alex/train/20240913_rc_brain_2_3_fos_neun_gfp-01_AcquisitionBlock8_pt8-Stitching-08_C0_Z0.ome_mask.tiff is a low contrast image
  return func(*args, **kwargs)


time : 106.74s 20240913_rc_brain_2_3_fos_neun_gfp-01_AcquisitionBlock8_pt8-Stitching-08_C0_Z0.ome


/home/bioimaging/miniconda3/envs/cellpose/lib/python3.10/site-packages/skimage/_shared/utils.py:328: UserWarning: /media/bioimaging/SWAP/Rita_Alex/train/20240913_rc_brain_2_3_fos_neun_gfp-01_AcquisitionBlock8_pt8-Stitching-08_C0_Z1.ome_mask.tiff is a low contrast image
  return func(*args, **kwargs)


time : 106.63s 20240913_rc_brain_2_3_fos_neun_gfp-01_AcquisitionBlock8_pt8-Stitching-08_C0_Z1.ome


/home/bioimaging/miniconda3/envs/cellpose/lib/python3.10/site-packages/skimage/_shared/utils.py:328: UserWarning: /media/bioimaging/SWAP/Rita_Alex/train/20240913_rc_brain_2_3_fos_neun_gfp-01_AcquisitionBlock8_pt8-Stitching-08_C0_Z2.ome_mask.tiff is a low contrast image
  return func(*args, **kwargs)


...

time : 118.13s 20240913_rc_brain_2_3_fos_neun_gfp-01_AcquisitionBlock8_pt8-Stitching-08_C3_Z3.ome


/home/bioimaging/miniconda3/envs/cellpose/lib/python3.10/site-packages/skimage/_shared/utils.py:328: UserWarning: /media/bioimaging/SWAP/Rita_Alex/train/20240913_rc_brain_2_3_fos_neun_gfp-01_AcquisitionBlock8_pt8-Stitching-08_C3_Z4.ome_mask.tiff is a low contrast image
  return func(*args, **kwargs)


time : 114.75s 20240913_rc_brain_2_3_fos_neun_gfp-01_AcquisitionBlock8_pt8-Stitching-08_C3_Z4.ome


/home/bioimaging/miniconda3/envs/cellpose/lib/python3.10/site-packages/skimage/_shared/utils.py:328: UserWarning: /media/bioimaging/SWAP/Rita_Alex/train/20240913_rc_brain_2_3_fos_neun_gfp-01_AcquisitionBlock8_pt8-Stitching-08_C3_Z5.ome_mask.tiff is a low contrast image
  return func(*args, **kwargs)


time : 101.97s 20240913_rc_brain_2_3_fos_neun_gfp-01_AcquisitionBlock8_pt8-Stitching-08_C3_Z5.ome


# Step 3: Cellpose Parameter Optimization (Optional)

## Why Parameter Optimization Matters

Different microscopy setups, cell types, and staining conditions may require parameter adjustment to achieve optimal segmentation results. This step helps you find the best Cellpose settings for your specific dataset.

## Key Parameters to Optimize

| Parameter | Description | Default | Range |
|-----------|-------------|---------|--------|
| `diameter` | Expected cell diameter in pixels | `None` (auto-detect) | Positive integer or `None` |
| `flow_threshold` | Quality threshold for optical flow | `0.4` | `0.0` - `1.0` |
| `cellprob_threshold` | Cell probability threshold | `0.0` | `-6.0` - `6.0` |

## When to Use This Tool

- **New cell types**: Different from standard training data
- **Different imaging conditions**: Unusual brightness, contrast, or resolution  
- **Poor segmentation results**: Default parameters aren't working well
- **Optimization needed**: Want to maximize accuracy for your specific dataset

## Workflow

1. Select a representative image from your dataset
2. Define parameter combinations to test
3. Run the optimization function
4. Compare cell counts and visual results
5. Update main pipeline with optimal parameters

**⚠️ Important**: Run this BEFORE the main segmentation in Step 2 to find optimal parameters for your dataset.

In [ ]:
# Parameter Testing Implementation
# Import additional libraries for parameter optimization
import matplotlib.pyplot as plt
from cellpose import models
import skimage.io as skio
import numpy as np
from pathlib import Path

def test_cellpose_parameters(image_path, param_combinations):
    """
    Test different parameter combinations on a single representative image
    
    Parameters:
    - image_path: Path to a representative image for testing
    - param_combinations: List of dictionaries with parameter combinations to test
    
    Returns:
    - Dictionary with results for each parameter combination
    """
    
    # Load the test image
    test_image = skio.imread(image_path)
    model = models.CellposeModel(pretrained_model='cpsam', gpu=True)
    
    results = {}
    
    print(f"Testing parameters on: {Path(image_path).name}")
    print("-" * 60)
    
    for i, params in enumerate(param_combinations):
        print(f"Testing combination {i+1}/{len(param_combinations)}: {params}")
        
        try:
            # Run segmentation with current parameters
            masks, flows, styles = model.eval(
                test_image,
                diameter=params.get('diameter', None),
                flow_threshold=params.get('flow_threshold', 0.4),
                cellprob_threshold=params.get('cellprob_threshold', 0.0),
                do_3D=False
            )
            
            # Count detected cells
            num_cells = len(np.unique(masks)) - 1  # Subtract 1 for background
            
            results[f"combo_{i+1}"] = {
                'parameters': params,
                'num_cells_detected': num_cells,
                'masks': masks
            }
            
            print(f"  → Detected {num_cells} cells")
            
        except Exception as e:
            print(f"  → Error: {e}")
            results[f"combo_{i+1}"] = {
                'parameters': params,
                'error': str(e)
            }
    
    return results

# Example usage - uncomment to run parameter optimization
"""
# Define parameter combinations to test
parameter_combinations = [
    {'diameter': None, 'flow_threshold': 0.4, 'cellprob_threshold': 0.0},   # Default
    {'diameter': None, 'flow_threshold': 0.6, 'cellprob_threshold': 0.0},   # Higher flow threshold
    {'diameter': None, 'flow_threshold': 0.3, 'cellprob_threshold': 0.0},   # Lower flow threshold
    {'diameter': None, 'flow_threshold': 0.4, 'cellprob_threshold': -1.0},  # Lower cell prob threshold
    {'diameter': None, 'flow_threshold': 0.4, 'cellprob_threshold': 1.0},   # Higher cell prob threshold
    {'diameter': 30, 'flow_threshold': 0.4, 'cellprob_threshold': 0.0},     # Fixed diameter
]

# Select a representative image for testing
test_image_path = "../slice_8/train/20240913_rc_brain_2_3_fos_neun_gfp-01_AcquisitionBlock8_pt8-Stitching-08_C0_Z0.ome.tif"

# Run parameter testing
if Path(test_image_path).exists():
    parameter_results = test_cellpose_parameters(test_image_path, parameter_combinations)
    
    # Display results summary
    print("\n" + "="*60)
    print("PARAMETER OPTIMIZATION RESULTS")
    print("="*60)
    
    for combo_name, result in parameter_results.items():
        if 'error' not in result:
            params = result['parameters']
            cells = result['num_cells_detected']
            print(f"{combo_name}: {cells} cells | {params}")
    
    print("\nRecommendation: Choose parameters that give reasonable cell counts")
    print("for your specific images and adjust the main pipeline accordingly.")
else:
    print(f"Test image not found: {test_image_path}")
    print("Update the path to point to one of your processed images.")
"""

print("Parameter optimization tools loaded.")
print("Uncomment the code above to test different parameter combinations on your images.")
